In [97]:
import numpy as np

In [98]:
class MyError(Exception):
    pass

In [99]:
 #Gauss–Jordan elimination
class Gauss_Jordan():
    __MAX_MATRIX_SIZE = 10
    __MIN_MATRIX_SIZE = 4
    
    def __init__(self):
       
       self._input_matrix = np.array([])
       self._size_input_matrix = self.__MIN_MATRIX_SIZE
       self._one_diag_matrix = np.array([])

    
    def _GenerateRandomMatrix(self, new_mat_size):

        if new_mat_size < self.__MIN_MATRIX_SIZE or new_mat_size > self.__MAX_MATRIX_SIZE:
            raise MyError(f"Invalid size for input matrix: {new_mat_size}. Size should be in interval [{self.__MIN_MATRIX_SIZE}, {self.__MAX_MATRIX_SIZE}]")
        
        self._size_input_matrix = new_mat_size

        self._input_matrix = np.zeros([self._size_input_matrix, self._size_input_matrix], dtype = int)

        for i in range(0, self._size_input_matrix):
            r = np.random.randint(100, size=self._size_input_matrix)
            self._input_matrix[i,:] = r

        print('A: ')
        print(self._input_matrix)

        print('one: ')
        self._one_diag_matrix = np.eye(self._size_input_matrix, dtype=int)
        print(self._one_diag_matrix)
    
    def G_J_solvetion(self, new_mat_size:int = __MIN_MATRIX_SIZE):
        
        self._GenerateRandomMatrix(new_mat_size)

        

    def TestInversionMatrix():
        pass
    

In [100]:

solvation = Gauss_Jordan()
solvation.GenerateRandomMatrix(5)

#size_input_matrix = 5
#empty_array = np.zeros([size_input_matrix, size_input_matrix], dtype=int)


A: 
[[78 65 63 65  5]
 [61  7 23 72 77]
 [41  0 42 82 10]
 [36 24 37 26 96]
 [93 33  6 55 92]]
one: 
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
